## Calculate ATO for TAZ centroid(s)

1. Run 1_setup_network.ipynb to create NetworkDataset_MM
2. Create OD Cost Matrix Layer
3. Add origin(s)
4. Add destinations
5. Solve
6. Join attributes to solved Lines layer
7. Weight HH and JOB by time decay
8. Sum

In [68]:
from arcgis.gis import GIS
import arcpy
import os

from arcgis.features import SpatialDataFrame
import pandas as pd
# from arcgis.features import GeoAccessor, GeoSeriesAccessor

def survey_weight(t):
    if t <= 3:
        return 1
    elif (t > 3) & (t <= 20):
        return -0.0382 * t + 1.1293
    elif t > 20:
        return 1/(1 + math.exp(0.1092 * t - 1.5604))
    else:
        return 0

# Update this
base_path = r"c:\wfrc\ato"

base_gdb = os.path.join(base_path, "ato.gdb")

In [69]:
# from 1_setup_network.ipynb

# baseline
arcpy.env.workspace = base_gdb
input_network_dataset = os.path.join(base_gdb, r"NetworkDataset\NetworkDataset_ND")
output_file = "driving_baseline"


# mod
#target_gdb =  os.path.join(base_path, "mod_drive.gdb")
#arcpy.env.workspace = target_gdb
#input_network_dataset = os.path.join(target_gdb, r"NetworkDataset\NetworkDataset_ND")
#output_file = "driving_mod"

mode = "Driving" # Driving | Transit

In [73]:
# Create OD Cost Matrix Layers
arcpy.na.MakeODCostMatrixAnalysisLayer(
    network_data_source = input_network_dataset, 
    layer_name = "OD Cost Matrix", 
    travel_mode = mode, 
    cutoff = 60.0,
    line_shape = "NO_LINES"
)

<Result 'OD Cost Matrix'>

In [75]:
# Add Origins
origin_options = "Name CO_TAZID #;TargetDestinationCount # #;CurbApproach # 0;Cutoff_Mins_BikeOnly # #;Cutoff_Mins_BikeTransitOnly # #;Cutoff_Mins_DriveOnly # #;Cutoff_Mins_PedOnly # #;Cutoff_Mins_PedTransitOnly # #;Cutoff_Mins_TransitOnly # #;Cutoff_Miles # #"

arcpy.na.AddLocations(
    in_network_analysis_layer = "OD Cost Matrix",
    sub_layer = "Origins", 
    in_table = os.path.join(base_path, r"shp\taz_wfrc.gdb\taz_centroids_snapped"), 
    field_mappings = origin_options, 
    search_tolerance = "10000 Meters", 
    search_criteria  = [["BikePedAuto","SHAPE"],
                        ["ConnectorNetwork","NONE"],
                       ["NetworkDataset_ND_Junctions","NONE"],
                       ["TransitRoutes","NONE"],
                       ["TransitStops","NONE"]], 
    match_type = "Priority",
    snap_to_position_along_network = "NO_SNAP", # input points are already snapped
    snap_offset = "0 Meters",
    exclude_restricted_elements = "EXCLUDE"
)

<Result 'OD Cost Matrix'>

In [76]:
# Add Destinations
arcpy.na.AddLocations(
    in_network_analysis_layer = "OD Cost Matrix",
    sub_layer = "Destinations", 
    in_table = os.path.join(base_path, r"shp\taz_wfrc.gdb\taz_centroids_snapped"), 
    field_mappings = "Name CO_TAZID #;CurbApproach # 0", 
    search_tolerance = "10000 Meters", 
    search_criteria  = [["BikePedAuto","SHAPE"],
                        ["ConnectorNetwork","NONE"],
                       ["NetworkDataset_ND_Junctions","NONE"],
                       ["TransitRoutes","NONE"],
                       ["TransitStops","NONE"]], 
    match_type = "Priority",
    snap_to_position_along_network = "NO_SNAP",
    snap_offset = "0 Meters",
    exclude_restricted_elements = "EXCLUDE"
)

<Result 'OD Cost Matrix'>

In [77]:
%%time
# Solve
arcpy.na.Solve("OD Cost Matrix", "SKIP", "TERMINATE", None, '')

Wall time: 6min 21s


id,value
0,a Layer object
1,true


In [78]:
%%time
od = pd.DataFrame.spatial.from_featureclass(r"OD Cost Matrix\Lines")

Wall time: 20.7 s


In [79]:
od.tail()

,ObjectID,Name,OriginID,DestinationID,DestinationRank,Total_Mins_BikeOnly,Total_Mins_BikeTransitOnly,Total_Mins_DriveOnly,Total_Mins_PedOnly,Total_Mins_PedTransitOnly,Total_Mins_TransitOnly,Total_Miles,SHAPE
5889350,5889351,491801 - 350980,2858,756,1579,None,None,59.980180,None,None,None,None,None
5889351,5889352,491801 - 350768,2858,598,1580,None,None,59.980286,None,None,None,None,None
5889352,5889353,491801 - 351008,2858,784,1581,None,None,59.980945,None,None,None,None,None
5889353,5889354,491801 - 351276,2858,1004,1582,None,None,59.985345,None,None,None,None,None
5889354,5889355,491801 - 492562,2858,1874,1583,None,None,59.992509,None,None,None,None,None


In [80]:
%%time
od['Origin_CO_TAZID'] = od['Name'].apply(lambda x: int(x.split(' - ')[0]))
od['Dest_CO_TAZID'] = od['Name'].apply(lambda x: int(x.split(' - ')[1]))

Wall time: 7.85 s


In [81]:
taz = pd.DataFrame.spatial.from_featureclass(os.path.join(base_gdb, "taz_ato"))

In [82]:
taz = taz[['CO_TAZID', 'HH_19', 'JOB_19', 'JOBAUTO_19', 'HHAUTO_19', 'JOBTRANSIT_19', 'HHTRANSIT_19']]

In [83]:
df = pd.merge(od, taz, left_on="Dest_CO_TAZID", right_on="CO_TAZID", )

In [84]:
df.head()

,ObjectID,Name,OriginID,DestinationID,DestinationRank,Total_Mins_BikeOnly,Total_Mins_BikeTransitOnly,Total_Mins_DriveOnly,Total_Mins_PedOnly,Total_Mins_PedTransitOnly,Total_Mins_TransitOnly,Total_Miles,SHAPE,Origin_CO_TAZID,Dest_CO_TAZID,CO_TAZID,HH_19,JOB_19,JOBAUTO_19,HHAUTO_19,JOBTRANSIT_19,HHTRANSIT_19
0,1,491896 - 491896,1,1,1,None,None,0.000000,None,None,None,None,None,491896,491896,491896,0.0,286.6,46951,37564,0,0
1,3417,492490 - 491896,2,1,1127,None,None,38.997650,None,None,None,None,None,492490,491896,491896,0.0,286.6,46951,37564,0,0
2,5687,492493 - 491896,3,1,1127,None,None,40.453651,None,None,None,None,None,492493,491896,491896,0.0,286.6,46951,37564,0,0
3,7943,492632 - 491896,4,1,1127,None,None,39.740380,None,None,None,None,None,492632,491896,491896,0.0,286.6,46951,37564,0,0
4,9111,491859 - 491896,5,1,32,None,None,8.768959,None,None,None,None,None,491859,491896,491896,0.0,286.6,46951,37564,0,0


In [85]:
# Weight outputs

if mode == 'Driving':
    df['travel_time'] = df['Total_Mins_DriveOnly']
elif mode == 'Transit':
    df['travel_time'] = df['Total_Mins_PedTransitOnly']

df['survey_weight'] = df['travel_time'].apply(lambda x: survey_weight(x))
df['survey_weight'] = df['survey_weight'].round(3)

df['weighted_jobs'] = df['survey_weight'] * df['JOB_19']
df['weighted_hh'] = df['survey_weight'] * df['HH_19']
df['weighted_jobs'] = round(df['weighted_jobs'])
df['weighted_hh'] = round(df['weighted_hh'])

df['ato'] = df['weighted_jobs'] + df['weighted_hh']

In [86]:
# write to disk
df[['Name', 'Origin_CO_TAZID', 'Dest_CO_TAZID', 'travel_time', 'survey_weight',
         'weighted_jobs', 'weighted_hh', 'ato']].to_csv(output_file + '.csv')

In [87]:
taz_summary = df.groupby('Origin_CO_TAZID').agg(
    jobs=pd.NamedAgg(column='weighted_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='weighted_hh', aggfunc=sum)
)
taz_summary['ato'] = taz_summary['jobs'] + taz_summary['hh']
taz_summary.to_csv(output_file + '_summary.csv')